#Модуль А

С помощью pandas.read_csv был прочитан предоставленный в задании датасет, даты были обработаны с помощью parse_dates, sep='|', т.к. такое разделение в основном файле

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('albums.csv', delimiter='|', parse_dates=['release_date'])

FileNotFoundError: ignored

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df['t_name0'].value_counts()

In [ ]:
df = df.drop(columns=['id', 'Unnamed: 0', 'name', 't_name0', 't_name1', 't_name2'])

In [ ]:
df.info()

In [ ]:
df['artists'].value_counts()

Удалим ненужные столбцы: Unnamed:0, и id являются уникальными индетификаторами, тоже самое можно сказать про name - для разделения на категории слишком много уникальных значений, а для языкового анализа физически не хватит времени на демоэкзамене. Тоже можно сказать про t_name. Имеет смысл рассмотреть artists, потому что популярность артиста определенно может влиять на популярность трека, однако универсальных объектов слишком много, оставим это на позже, а теперь просмотрим количество пропусков:

In [ ]:
df.isna().sum()

удалим все данные, не имеющие нашего целевого признака: popularity, т.к. заполнение его средним значением может пагубно повлиять на качество модели

In [ ]:
df = df.dropna(subset=['popularity'])

In [ ]:
df.isna().sum()

In [ ]:
df.info()

Имеется относительно небольшое число данных, в которых отсутствует t_*имя атрибута*0, их в целом тоже можно удалить, т.к. без него мы в целом не знаем что-либо о треках из альбома/сингла.

In [ ]:
df = df.dropna(subset=['t_val0', 't_sig0', 't_live0','t_tempo0', 't_ins0', 't_acous0','t_key0', 't_dance0'])

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

Тут вскрывается один минус датасета, из 160 тысяч объектов, где-то у трети нет t_*имя атрибута*1 или 2. По-хорошему, нужно построить различные модели, чтобы оценить различные способы заполнения, однако времени на это нет и мы только в Модуле А, поэтому выбираем один из трех вариантов: заполнить нулями, заполнить средним и удалить данные с пропусками. для этих столбцов выберем заполнение средним и проверим по нескольким графикам не сменилось ли распределение.

In [ ]:
import seaborn as sns
def test_density_fillmean(df, col: str):
    sns.histplot(df[col], kde=True, stat='density')
    sns.histplot(df[col].fillna(df[col].mean()), kde=True, stat='density')

In [ ]:
test_density_fillmean(df,'t_val1')

Видим, что распределение изменилось драматически, при заполнении нулём:

In [ ]:
import seaborn as sns
def test_density_fillzero(df, col: str):
    sns.histplot(df[col], kde=True, stat='density')
    sns.histplot(df[col].fillna(0), kde=True, stat='density')

In [ ]:
test_density_fillzero(df,'t_val2')

Тоже самое.

In [ ]:
import missingno as msno
msno.matrix(df)

In [ ]:
df[['t_dur1', 't_val1', 't_dance1']].isnull().corr()

Визуально видно, да и по корреляции, что пропуски находятся в одних и тех же местах. Т.е. появляется вариант удалить данные с пропусками, но это УДАЛИТ ТРЕТЬ ДАННЫХ и полностью уберёт синглы, остаётся заполнить все значения нулями, т.к на проверку всех атрибутов времени физически нет :(
    

In [ ]:
for column in df.columns:
    if df[column].isna().sum() > 0:
        df[column] = df[column].fillna(0)

In [ ]:
df.isna().sum()

Пропуски заполнили, теперь переходим к колонке artists. По сути этот столбец конкатинирует в себе несколько артистов и по-хорошему нужно поделить его на несколько столбцов, однако мы не знаем максимальное количество аритстов в релизе(их может быть 10), а также это приведёт к огромному количеству пропусков, так что я решился просто убрать спецсимволы, а затем преобразовать этот признак с помощью TargetEncoder.

In [ ]:
!pip install category-encoders

In [ ]:
import category_encoders as ce

In [ ]:
df['artists'] = df['artists'].str.replace('\W','')

In [ ]:
te = ce.TargetEncoder(cols=['artists'])
X = df.drop(columns=['popularity'])
Y = df['popularity']
X = te.fit_transform(X, Y)

In [ ]:
X

In [ ]:
df = pd.concat([X, Y], axis=1)

In [ ]:
df['artists'].corr(df['popularity'])

In [ ]:
import numpy as np

Видим, что корреляция огромная и в дальнейшем возможно придётся избавляться от этого признака, но что имеем, то имеем. Переходим к преобразованию даты на столбцы.

In [ ]:
date = [i.split('-') for i in df['release_date']]
date

In [ ]:
def column(matrix, i):
    return [row[i]  for row in matrix]

ВИДИМ ЧТО В НЕКОТОРЫХ ДАТАХ ОКАЗЫВАЕТСЯ ТОЛЬКО ГОД, ПОЭТОМУ сохраняем только год.

In [ ]:
df['year'] = column(date, 0)
df.drop(['release_date'], axis=1, inplace=True)
df

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
num_col = []
cat_col = []
for col in df.columns:
    if df.dtypes[col] in ('int64', 'float64'):
        num_col.append(col)
    else:
        cat_col.append(col)
num_col

Строим тепловую карту, масштабируем с помощью MinMax, т.к. не все данные нормальные(было видно ранее) и строим pairplot

In [ ]:
sns.heatmap(data=df[num_col].corr(), vmin=-1, vmax=1, fmt='.2f', cmap="crest")

Видим, что коррелируют с собой схожие атрибуты с индексами _1 и _2, что логично и как ранее было видно artists с popularity



In [ ]:
num_col.remove('popularity')
num_col.remove('year')

In [ ]:
num_col

In [ ]:
df['year'] = df['year'].astype(int)

In [ ]:
MinMax = MinMaxScaler()
df[num_col] = MinMax.fit_transform(df[num_col])
df[num_col]

Время поджимало, поэтому такой формат: все признаки и их корреляция с целевым

In [ ]:
for i in df.columns:
    df.plot.scatter (x = 'popularity', y = i)

In [ ]:
df.to_csv('preprocessed.csv', index=False)

#Модуль B

Рассмотрим значения признаков с помощью дерева

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
X = df.drop(columns=['popularity'])
y = df['popularity']
DT = DecisionTreeRegressor()
features = DT.fit(X,y).feature_importances_

In [ ]:
import matplotlib.pyplot as plt
plt.barh(X.columns, features)

Как видим, как и предполагалось значение артистов слишком велико и необходимо убрать этот столбец из датасета F :(. Построим дерево заново без данного признака

In [ ]:
df = df.drop(columns=['artists'])

In [ ]:
X = df.drop(columns=['popularity'])
y = df['popularity']
DT = DecisionTreeRegressor()
features = DT.fit(X,y).feature_importances_

In [ ]:
plt.barh(X.columns, features)

In [ ]:
X.shape

Теперь нет выбивающихся признаков и можно переходить к другим способам, будем тестировать модели на BaggingRegressor. Делить выборку будем с помощью train_test_split. Тестовую выборку оставим 0.6,чтобы не тратить много времени на тестирование уменьшения признаков. random_state=22, т.к. моё любимое число :)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error,r2_score
from math import sqrt

In [ ]:
def test(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=22,shuffle=True)
    model = BaggingRegressor().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
    print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
    print(f'R^2: {r2_score(y_test, y_pred)}')
    return model

In [ ]:
from sklearn.feature_selection import SelectKBest
for i in range(1, 11):
    print(i)
    X_new = SelectKBest(k=i).fit_transform(X, y)
    test(X_new, y)
    print('________________________________')


Видим, что качество модели почти перестаёт ухудшаться после 4-5 признаков, выведем признаки в этих количествах



In [ ]:
from sklearn.feature_selection import SelectKBest
for i in range(4, 7):
    print(i)
    KBest = SelectKBest(k=i)
    X_new = KBest.fit_transform(X, y)
    print(KBest.get_feature_names_out())
    test(X_new, y)
    print('________________________________')

In [ ]:
from sklearn.decomposition import PCA
for i in range(1, 11):
    print(i)
    X_new = PCA(n_components=i).fit_transform(X, y)
    test(X_new, y)
    print('________________________________')

Видим, что оптимальное количество признаков 4-6

Видим, что PCA показал себя гораздо лучше, теперь попробуем RFE.

In [ ]:
from sklearn.feature_selection import RFE
tree = DecisionTreeRegressor().fit(X, y)
for i in range(4, 7):
    print(i)
    rfe = RFE(estimator=tree, n_features_to_select=i, step=1).fit(X, y)
    X_new = rfe.transform(X)
    test(X_new, y)
    print('________________________________')
rfe = RFE(estimator=tree, n_features_to_select=4, step=1).fit(X, y)
X_rfe = pd.DataFrame(rfe.transform(X), columns=rfe.get_feature_names_out())
X_rfe

RFE показал результат, схожий с PCA, но при этом сохранил изначальные признаки, поэтому оставим его.

In [ ]:
rfe = RFE(estimator=tree, n_features_to_select=4, step=1).fit(X, y)
X_rfe = pd.DataFrame(rfe.transform(X), columns=rfe.get_feature_names_out())
X_rfe

In [ ]:
rfe_test = RFE(estimator=tree, n_features_to_select=10, step=1).fit(X, y)
X_rfe_test = pd.DataFrame(rfe_test.transform(X), columns=rfe_test.get_feature_names_out())
test(X_rfe_test, y)

Проверил, что при сильном увеличении размерности результат не сильно меняется, поэтому оставляю 4

In [ ]:
df_visual = pd.concat([X_rfe, y], axis=1)

In [ ]:
sns.heatmap(data=df_visual.corr(), vmin=-1, vmax=1, fmt='.2f', cmap="crest")

In [ ]:
df_visual.boxplot(column='popularity')

Рассмотрим визуализации: коррелирования с целевым признаком нет, поэтому линейные модели вряд-ли покажут себя хорошо, целевой признак сам по себе
нормально распределён в диапазоне от 30 до 70, при этом имеет максимум 100 и минимум 0.

Сделаем новую разбивку данных, на сей раз, в отличие от теста, оставим на обучение 0.7 данных, чтобы модели показали наилучший результат.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_rfe, y, test_size=0.3, random_state=22,shuffle=True)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
params = {'max_depth' : np.arange(20, 100, 20),
}
model = GridSearchCV(DecisionTreeRegressor(),params).fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {r2_score(y_test, y_pred)}')

Дерево показало себя +- неплохо, однако неидеально. Для любопытства посмотрим тоже само на PCA

In [ ]:
X_PCA = PCA(n_components=5).fit_transform(X, y)

In [ ]:
X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(X_PCA, y, test_size=0.3, random_state=22,shuffle=True)

In [ ]:
params = {'max_depth' : np.arange(20, 100, 20),
}
model = GridSearchCV(DecisionTreeRegressor(),params).fit(X_new_train, y_new_train)
y_pred = model.predict(X_new_test)
print(f'MAE: {mean_absolute_error(y_new_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_new_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_new_test, y_pred))}')
print(f'MAPE: {(mean_absolute_percentage_error(y_new_test, y_pred))}')
print(f'R^2: {r2_score(y_new_test, y_pred)}')

Видим, что разница небольшая, даже немного хуже, так что оставляем признаки из RFE

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
params = {'alpha' : np.arange(0, 1, 0.1),
}
model = GridSearchCV(Ridge(),params).fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {r2_score(y_test, y_pred)}')

Как и предполагалось линейная модель показала себя плохо и не подходит при данном уменьшении размерности.

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
model = CatBoostRegressor().fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {r2_score(y_test, y_pred)}')

CatBoost показал себя неплохо, но не лучше чем дерево.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
model = RandomForestRegressor().fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {r2_score(y_test, y_pred)}')

In [ ]:
params = {'n_estimators' : np.arange(50, 80, 10),
           'max_depth': np.arange(10, 25, 5)}
model = RandomizedSearchCV(RandomForestRegressor(), params).fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {r2_score(y_test, y_pred)}')

RandomizedSearch не успел :(

In [ ]:
choosen_model = RandomForestRegressor().fit(X_train, y_train)

In [ ]:
import pickle

In [ ]:
with open('model.pkl', 'wb') as dump_out:
    pickle.dump(choosen_model, dump_out)

Случайный лес показал себя лучше всех, поэтому в следующих этапах будем использовать RandomForest, его и сохраним

#Модуль C

In [ ]:
!pip install -q streamlit
!pip install -q streamlit-option-menu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.2/799.2 kB 21.4 MB/s eta 0:00:00


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.144s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



Создадим MinMaxPreprocesser, обученный на изначальных данных, для использования в Dash

In [ ]:
df = pd.read_csv('albums.csv', delimiter='|', parse_dates=['release_date'])
df = df.dropna(subset=['t_val0', 't_sig0', 't_live0','t_tempo0', 't_ins0', 't_acous0','t_key0', 't_dance0'])
df = df.dropna(subset=['popularity'])
needed_data = df[["total_tracks", "t_dur0", "t_speech0", "t_acous0"]]
MinMax = MinMaxScaler()
MinMax.fit(needed_data)
with open('MinMax.pkl', 'wb') as dump_out:
    pickle.dump(MinMax, dump_out)

In [ ]:
df = pd.read_csv('albums.csv', delimiter='|', parse_dates=['release_date'])
df.head(10).to_csv('test.csv', index=False, sep='|')

In [ ]:
%%writefile app.py

import pickle
import seaborn as sns
import streamlit as st
import pandas as pd
from streamlit_option_menu import option_menu
import matplotlib.pyplot as plt
import numpy as np

with open('model.pkl', 'rb') as f:
    random_forest = pickle.load(f)

with open('MinMax.pkl', 'rb') as f:
    num_preprocessor = pickle.load(f)

def preprocessing(df):
    df = df.dropna(subset=['total_tracks', 't_dur0', 't_speech0', 't_acous0'])
    df[['total_tracks', 't_dur0', 't_speech0', 't_acous0']] = num_preprocessor.transform(df[['total_tracks', 't_dur0', 't_speech0', 't_acous0']])
    return df[['total_tracks', 't_dur0', 't_speech0', 't_acous0']]

def upload():
    uploaded_file = st.file_uploader("Выберите файл .csv (delimiter = '|')")

    if uploaded_file:
        upload_data = pd.read_csv(uploaded_file, sep="|")
        upload_data = preprocessing(upload_data)

    if st.button("Получить предсказание", key='1'):
        if not uploaded_file:
            st.warning('Сначала нужно загрузить данные!', icon="⚠️")
        else:
            y_pred = random_forest.predict(upload_data)
            upload_data['popularity'] = y_pred
            st.write(upload_data)
            st.download_button("Сохранить результат", upload_data.to_csv(index=False, encoding='utf-8', sep='|'), "file.csv", "text/csv", key='download-csv')

def input():
    edited_df = st.data_editor(pd.DataFrame([{
        "total_tracks": "",
        "t_dur0": "",
        "t_speech0": "",
        "t_acous0": "",
    }]), num_rows="dynamic", use_container_width=False, width = 1000)

    if st.button("Получить предсказание", key='2'):
        try:
            edited_df = preprocessing(edited_df)
            y_pred = random_forest.predict(edited_df)
            edited_df['popularity'] = y_pred
            st.write(edited_df)
            st.download_button("Сохранить результат", edited_df.to_csv(index=False, encoding='utf-8',sep='|'), "file.csv", "text/csv", key='download-csv')
        except:
            st.warning('Некорректные данные!', icon="⚠️")

def prediction():
    tab1, tab2 = st.tabs(["Загрузка файла .csv", "Ручной ввод значений"])

    with tab1:
        upload()

    with tab2:
        input()

INFO = r"""
# Добро пожаловать в приложение для предсказывания популярности вашего трека!

## Описание входных данных (объекты)

Для предсказания вам необходимы не все данные, а только перечисленные ниже. Все они являются числовыми

|  Название столбца  | Описание                                   |
| ------------------ | ------------------------------------------ |
| total_tracks       | Количество треков в релизе                 |
| t_dur0             | Длительность 1 трека в релизе  (в мс)      |
| t_speech0          | Соотношение слов в треке(от 0 до 1)              |
| t_acous0           | Насколько звучание близко к акустике(от 0 до 1)  |


## Описание выходных данных (предсказания)

Предсказываться будет значение $popularity \in $ ($0, 100$) - потенциальная популяность вашего трека.

Таким образом, решается задача регресссии.

## Модель

В предсказании используется модель RandomForest, которая показала наилучшие результаты на выделенном подпространстве признаков.

## Визуализации

Предоставлена визуализация heatmap усеченного пространства признаков.

"""

def data_description():
    st.markdown(INFO)

def visualisation():
    df = pd.read_csv('C[03]_A_preprocessed.csv', delimiter=',', encoding = 'utf-8')
    df_col = df[['total_tracks', 't_dur0', 't_speech0', 't_acous0', 'popularity']]
    fig, ax = plt.subplots()
    sns.heatmap(df_col.corr(), ax=ax)
    st.write('Тепловая карта признаков')
    st.pyplot(fig)


menu = {
    "app":
    {
        "Описание данных": data_description,
        "Предсказание": prediction,
        "Визуализация": visualisation,
    }
}

if __name__ == "__main__":
    with st.sidebar:
            selected = option_menu("Меню", ["Описание данных", 'Визуализация', 'Предсказание'],
                icons=['info-circle', 'bar-chart', 'tag'], menu_icon="cast", default_index=0)

    menu["app"][selected]()

Writing app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com

In [ ]:
!npx localtunnel --port 8501